## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns={'EIN','NAME','SPECIAL_CONSIDERATIONS'})
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_types = pd.DataFrame(application_df['APPLICATION_TYPE'].value_counts())
application_types=application_types.reset_index().rename(columns={'index': 'APPLICATION_TYPE'})
application_types

,APPLICATION_TYPE,count
0,T3,27037
1,T4,1542
2,T6,1216
3,T5,1173
4,T19,1065
5,T8,737
6,T7,725
7,T10,528
8,T9,156
9,T13,66


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_types[application_types['count'] < 100]['APPLICATION_TYPE']
application_types_to_replace

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classifications = application_df['CLASSIFICATION'].value_counts()
classifications

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classifications_1 = classifications[classifications > 1]
classifications_1

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classifications[classifications < 200].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: count, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df,dtype=float)


In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL']
X = application_df.drop(columns='IS_SUCCESSFUL')
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,stratify=y)


In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## First Attempt to Optimize the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
n_i_f = len(X_train.columns)
h_n_l_1 = 80
h_n_l_2 = 60
h_n_l_3 = 40


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=h_n_l_1, activation="relu", input_dim=n_i_f))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=h_n_l_2, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=h_n_l_3, activation="swish"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

c:\Users\caroj\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 80)             │         3,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 60)             │         4,860 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 40)             │         2,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            41 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,941 (42.74 KB)

 Trainable params: 10,941 (42.74 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
fit_model = nn.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 807us/step - accuracy: 0.7054 - loss: 0.5828
Epoch 2/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 774us/step - accuracy: 0.7257 - loss: 0.5546
Epoch 3/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 758us/step - accuracy: 0.7298 - loss: 0.5499
Epoch 4/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 767us/step - accuracy: 0.7290 - loss: 0.5537
Epoch 5/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 863us/step - accuracy: 0.7298 - loss: 0.5498
Epoch 6/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 800us/step - accuracy: 0.7338 - loss: 0.5478
Epoch 7/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 879us/step - accuracy: 0.7282 - loss: 0.5501
Epoch 8/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7329 - loss: 0.5451
Epoch 9/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7379 - loss: 0.5399
Epoch 10/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 926us/step - accuracy: 0.7297 - loss: 0.5476
Epoch 11/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 922us/step - accuracy: 0.7338 - loss: 0.5387
Epoch 12/150
804/804 ━━

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 1ms/step - accuracy: 0.7307 - loss: 0.5735
Loss: 0.5734627842903137, Accuracy: 0.7307288646697998


## Second Attempt to Optimize the Model

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
n_i_f = len(X_train.columns)
h_n_l_1 = 100
h_n_l_2 = 80 
h_n_l_3 = 40
h_n_l_4 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=h_n_l_1, activation="elu", input_dim=n_i_f))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=h_n_l_2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=h_n_l_3, activation="tanh"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=h_n_l_4, activation="swish"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 100)            │         4,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 80)             │         8,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 40)             │         3,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 20)             │           820 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,661 (65.08 KB)

 Trainable params: 16,661 (65.08 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 938us/step - accuracy: 0.7061 - loss: 0.5813
Epoch 2/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 964us/step - accuracy: 0.7251 - loss: 0.5569
Epoch 3/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 960us/step - accuracy: 0.7323 - loss: 0.5489
Epoch 4/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7322 - loss: 0.5502
Epoch 5/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7374 - loss: 0.5444
Epoch 6/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7307 - loss: 0.5501
Epoch 7/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7317 - loss: 0.5494
Epoch 8/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7295 - loss: 0.5491
Epoch 9/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7380 - loss: 0.5421
Epoch 10/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7317 - loss: 0.5460
Epoch 11/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7333 - loss: 0.5438
Epoch 12/200
804/804 ━━━━━━━━━━━━━━

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 1ms/step - accuracy: 0.7284 - loss: 0.5628
Loss: 0.5628423094749451, Accuracy: 0.728396475315094


## Third Attempt to Optimize the Model

In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
n_i_f = len(X_train.columns)
h_n_l_1 = 40
h_n_l_2 = 30 
h_n_l_3 = 20
h_n_l_4 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=h_n_l_1, activation="relu", input_dim=n_i_f))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=h_n_l_2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=h_n_l_3, activation="tanh"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=h_n_l_4, activation="swish"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 40)             │         1,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 30)             │         1,230 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 20)             │           620 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,871 (15.12 KB)

 Trainable params: 3,871 (15.12 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [22]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 950us/step - accuracy: 0.6955 - loss: 0.5967
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 895us/step - accuracy: 0.7284 - loss: 0.5568
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 889us/step - accuracy: 0.7273 - loss: 0.5535
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 916us/step - accuracy: 0.7335 - loss: 0.5519
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 960us/step - accuracy: 0.7316 - loss: 0.5486
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 939us/step - accuracy: 0.7367 - loss: 0.5435
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 896us/step - accuracy: 0.7352 - loss: 0.5433
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 986us/step - accuracy: 0.7323 - loss: 0.5447
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 972us/step - accuracy: 0.7337 - loss: 0.5436
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 949us/step - accuracy: 0.7326 - loss: 0.5452
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 942us/step - accuracy: 0.7321 - loss: 0.5486
Epoch 12/100
804/80

In [23]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 1ms/step - accuracy: 0.7300 - loss: 0.5597
Loss: 0.5596678256988525, Accuracy: 0.7300291657447815


In [24]:
nn.save('AlphabetSoupCharity_Optimization.h5')